In [1]:
import os

In [2]:
if os.uname().nodename == 'lighthouse':
    root_path = '/home/andbro/'
    data_path = '/home/andbro/kilauea-data/'
    archive_path = '/home/andbro/freenas/'
    bay_path = '/home/andbro/ontap-ffb-bay200/'
    lamont_path = '/home/andbro/lamont/'
elif os.uname().nodename == 'kilauea':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'
    bay_path = '/import/ontap-ffb-bay200/'
    lamont_path = '/lamont/'
elif os.uname().nodename in ['lin-ffb-01', 'ambrym', 'hochfelln']:
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'
    bay_path = '/import/ontap-ffb-bay200/'
    lamont_path = '/lamont/'

In [3]:

class sixdegrees():

    from obspy import UTCDateTime, Stream

    def __init__(self, conf):

        self.config = conf

        # define startime
        if 'tbeg' in conf.keys():
            self.tbeg = UTCDateTime(conf['tbeg'])
        else:
            print("-> no starttime given!")

        # define endtime
        if 'tend' in conf.keys():
            self.tend = UTCDateTime(conf['tend'])
        else:
            print("-> no starttime given!")

        # set verbose (print information)
        if 'verbose' in conf.keys():
            self.verbose = conf['verbose']
        else:
            self.verbose = False

        # seed id of stream
        if 'seed' in conf.keys():
            self.net, self.sta, self.loc, self.cha = conf['seed'].split('.')
        else:
            self.net, self.sta, self.loc, self.cha = "XX.XXXX..".split('.')

        # seed id of rotation stream
        if 'rot_seed' in conf.keys():
            self.rot_seed = conf['rot_seed']
        else:
            print("-> no rotation seed id given!")

        # seed id of translation stream
        if 'tra_seed' in conf.keys():
            self.tra_seed = conf['tra_seed']
        else:
            print("-> no translation seed id given!")

        # station coordinates
        if 'station_lon' in conf.keys() and 'station_lat' in conf.keys():
            self.station_longitude = conf['station_lon']
            self.station_latitude = conf['station_lat']
        else:
            print("-> no station coordinates given!")

        # define project name
        if 'project' in conf.keys():
            self.project = conf['project']
        else:
            self.project = "test"

        # define working directory
        if 'workdir' in conf.keys():
            self.workdir = conf['workdir']
        else:
            self.workdir = "./"

        # define directory for output data
        if 'path_to_data_out' in conf.keys():
            self.path_to_data_out = conf['path_to_data_out']
        else:
            self.path_to_data_out = self.workdir+"output/"

        # path to SDS file structure for rotation data
        if 'path_to_sds_rot' in conf.keys():
            self.rot_sds = conf['path_to_sds_rot']
        else:
            print("-> no path to SDS file structure for rotation data given!")

        # path to SDS file structure for translaton data
        if 'path_to_sds_tra' in conf.keys():
            self.tra_sds = conf['path_to_sds_tra']
        else:
            print("-> no path to SDS file structure for translaton data given!")

        # path to translation station inventory
        if 'path_to_inv_tra' in conf.keys():
            self.tra_inv = conf['path_to_inv_tra']
        else:
            print("-> no path to translation station inventory given!")

        # path to rotation station inventory
        if 'path_to_inv_rot' in conf.keys():
            self.rot_inv = conf['path_to_inv_rot']
        else:
            print("-> no path to rotation station inventory given!")


        self.tra_output = "ACC"


    # ____________________________________________________

    def attributes(self):
        return [a for a in dir(self) if not a.startswith('__')]

    def check_path(self, dir_to_check):

        from os import path, mkdir

        if not path.isdir(dir_to_check):
            mkdir(dir_to_check)
            sixdegrees.check_path(self, dir_to_check)
        else:
            print(f" -> {dir_to_check} exists")

    def get_rotation(self):
        return self.st.select(channel="*J*")

    def get_translation(self):
        return self.st.select(channel="*H*")

    def read_from_sds(path_to_archive, seed, tbeg, tend, data_format="MSEED"):
        """
        VARIABLES:
         - path_to_archive
         - seed
         - tbeg, tend
         - data_format

        DEPENDENCIES:
         - from obspy.core import UTCDateTime
         - from obspy.clients.filesystem.sds import Client

        OUTPUT:
         - stream

        EXAMPLE:
        >>> st = __read_sds(path_to_archive, seed, tbeg, tend, data_format="MSEED")
        """

        import os
        from obspy.core import UTCDateTime, Stream
        from obspy.clients.filesystem.sds import Client

        tbeg, tend = UTCDateTime(tbeg), UTCDateTime(tend)

        if not os.path.exists(path_to_archive):
            print(f" -> {path_to_archive} does not exist!")
            return

        ## separate seed id
        net, sta, loc, cha = seed.split('.')

        ## define SDS client
        client = Client(path_to_archive, sds_type='D', format=data_format)

        ## read waveforms
        try:
            st = client.get_waveforms(net, sta, loc, cha, tbeg, tend, merge=-1)
        except:
            print(f" -> failed to obtain waveforms!")
            st = Stream()

        return st

    def load_data(self, t1, t2):
        '''
        Load data for translation and rotaion as obspy stream
        '''

        from obspy import Stream, UTCDateTime

        st0 = Stream()

        t1, t2 = UTCDateTime(t1), UTCDateTime(t2)
        print(self.tra_sds, self.tra_seed)
        try:
            # read translational data
            tra = sixdegrees.read_from_sds(self.tra_sds, self.tra_seed, t1-1, t2+1)

        except Exception as e:
            print(f" -> loading translation data failed!")
            if self.verbose:
                print(e)

        if self.verbose:
            print(tra)

        try:
            tra = tra.detrend("linear")

            tra = tra.filter('highpass', freq=0.005, zerophase=True)

            # remove response
            tra = tra.remove_response(self.tra_inv, output=self.tra_output)

            st0 += tra

        except Exception as e:
            print(f" -> removing response failed!")
            if self.verbose:
                print(e)

        try:
            # read rotational data
            rot = sixdegrees.read_from_sds(self.rot_sds, self.rot_seed, t1-1, t2+1)

            st0 += rot

        except Exception as e:
            print(f" -> loading rotation data failed!")
            if self.verbose:
                print(e)

        if self.verbose:
            print(rot)

        # check if stream has correct length
        if len(st0) < 6:
            print(f" -> missing stream data")

       # check if merging is required
        if len(st0) > 6:
            st0 = st0.merge(method=1, fill_value=0)

        # change seed id
        for tr in st0:
            tr.stats.network = self.net
            tr.stats.station = self.sta
            tr.stats.location = self.loc

        # assign stream to object
        self.st = st0

    def trim_stream(self, set_common=True, set_interpolate=False):
        '''
        Trim a stream to common start and end times (and interpolate to common times)
        '''

        from numpy import interp, arange

        def __get_size(st0):
            return [tr.stats.npts for tr in st0]

        # get size of traces
        n_samples = __get_size(self.st)

        # check if all traces have same amount of samples
        if not all(x == n_samples[0] for x in n_samples):
            print(f" -> stream size inconsistent: {n_samples}")

            # if difference not larger than one -> adjust
            if any([abs(x-n_samples[0]) > 1 for x in n_samples]):

                # set to common minimum interval
                if set_common:
                    _tbeg = max([tr.stats.starttime for tr in self.st])
                    _tend = min([tr.stats.endtime for tr in self.st])
                    self.st = self.st.trim(_tbeg, _tend, nearest_sample=True)
                    print(f"  -> adjusted: {__get_size(self.st)}")

                    if set_interpolate:
                        _times = arange(0, min(__get_size(self.st)), self.st[0].stats.delta)
                        for tr in self.st:
                            tr.data = interp(_times, tr.times(reftime=_tbeg), tr.data)
            else:
                # adjust for difference of one sample
                for tr in self.st:
                    tr.data = tr.data[:min(n_samples)]
                print(f"  -> adjusted: {__get_size(self.st)}")

    def get_time_intervals(self, tbeg=None, tend=None, interval_seconds=3600, interval_overlap=0):
        '''
        Obtain time intervals
        '''

        from obspy import UTCDateTime

        if tbeg is None:
            tbeg = self.tbeg
        else:
            tbeg = UTCDateTime(tbeg)

        if tend is None:
            tend = self.tend
        else:
            tend = UTCDateTime(tend)

        times = []
        t1, t2 = tbeg, tbeg + interval_seconds

        while t2 <= tend:
            times.append((t1, t2))
            t1 = t1 + interval_seconds - interval_overlap
            t2 = t2 + interval_seconds - interval_overlap

        self.time_intervals = times
        return times

    def get_octave_bands(self, fmin=None, fmax=None, faction_of_octave=1, plot=False):
        '''
        Computing octave frequency bands

        Arguments:
            - fmin:    (float) minimum center frequency
            - fmax:    (float) maximum center frequency
            - fraction_of_octave:    (int) octave fraction (e.g. [1] = octaves, 3 = third octaves, 12 = 12th octaves)
            - plot:    (bool) show frequency bands

        Example:

        >>> flower, fupper, fcenter = __get_octave_bands(f_min, f_max, fband_type="octave", plot=False)
        '''

        import matplotlib.pyplot as plt
        from acoustics.octave import Octave
        from numpy import array

        if fmin is None:
            fmin = self.fmin
        if fmax is None:
            fmax = self.fmax

        # avoid fmin = zero
        if fmin == 0:
            print(f" -> set fmin to 1e-10")
            fmin = 1e-10

        f_lower, f_upper, f_center = [], [], []

        # compute f-bands
        _octaves = Octave(fraction=faction_of_octave,
                          interval=None,
                          fmin=fmin,
                          fmax=fmax,
                          unique=False,
                          reference=1000.0
                         )

        # store for object
        self.f_center = array(_octaves.center)
        self.f_lower = array(_octaves.lower)
        self.f_upper = array(_octaves.upper)

        # checkup plot
        if plot:
            plt.figure(figsize=(15, 5))
            for fl, fc, fu in zip(self.f_lower, self.f_center, self.f_upper):
                plt.axvline(fu, color="r")
                plt.axvline(fl, color="r")
                plt.axvline(fc, ls="--")
                plt.axvline(fmin, color="g")
                plt.axvline(fmax, color="g")
                plt.xscale("log")
            plt.show();

        return self.f_lower, self.f_upper, self.f_center

    def interpolate_nan(array_like):
        '''
        interpolate NaN values in array linearly
        '''

        from numpy import isnan, interp

        array = array_like.copy()

        nans = isnan(array)

        def get_x(a):
            return a.nonzero()[0]

        array[nans] = interp(get_x(nans), get_x(~nans), array[~nans])

        return array

    def store_as_pickle(self, obj, name):

        import os, pickle

        ofile = open(name+".pkl", 'wb')
        pickle.dump(obj, ofile)

        if os.path.isfile(name+".pkl"):
            print(f" -> stored: {name}.pkl")

    def get_event_info(self, station_lat=None, station_lon=None, event_time=None, time_offset=20, fdsn_client="USGS"):
        '''
        Extract information of event from catalog
        '''

        from obspy.clients.fdsn import Client
        from obspy.geodetics.base import gps2dist_azimuth

        # get event if not provided
        if event_time is None:
            print(" -> provide event_time !!!")
            return
        else:
            events = Client(fdsn_client).get_events(starttime=event_time-time_offset, endtime=event_time+time_offset)
            if len(events) > 1:
                print(f" -> {len(events)} events found!!!")
                print(events)
                return

            elif len(events) > 0:
                event = events[0]
                print(f" ->  no event found!!!")
                return

        # event location from event info
        self.source_latitude = event.origins[0].latitude
        self.source_longitude = event.origins[0].longitude

        # compute paremeters
        if None in [station_lat, station_lon]:
            dist, az, baz = gps2dist_azimuth(
                                             self.source_latitude, self.source_longitude,
                                             self.station_latitude, self.station_lonitude,
                                            )
        else:
            dist, az, baz = gps2dist_azimuth(
                                             self.source_latitude, self.source_longitude,
                                             station_lat, station_lon,
                                            )

        self.baz_catalog = baz
        self.az_catalog = az
        self.epicentral_distance = dist

        if self.verbose:
            print(f" -> Epicentral distance [m]:       {round(self.epicentral_distance, 1)}")
            print(f" -> Theoretical azimuth [deg]:     {round(self.az_catalogbaz, 1)}")
            print(f" -> Theoretical backazimuth [deg]: {round(self.baz_catalogbaz, 1)}")

    def compute_backazimuth(self, wave_type="love", fmin=None, fmax=None, out=False):

        '''
        This method estimates a backazimuth for either
         - Love waves (vertical rotation rate and transverse acceleration) or
         - Rayleigh waves (vertical acceleration and transverse rotation rate)
        and compares it to a theoretical backazimuth computed from the source and receiver location.

        RETURN:
        - out:  dictionary with output data. Keywords:
                'baz_mesh', 'baz_corr', 'baz_theo', 'acc_transverse', 'acc_radial', 'rot_vertical', 'event'

        >>> out = __compute_backazimuth(st_acc, st_rot, config, wave_type="love", event=None, plot=True, show_details=False)

        '''

        from numpy import ones, arange, linspace, asarray, array, meshgrid, nan, shape
        from numpy import arctan, pi, linspace, cov, argsort, corrcoef, correlate, zeros
        from numpy.linalg import eigh
        from obspy.signal.cross_correlation import correlate, xcorr_max
        from obspy.signal.rotate import rotate_ne_rt

        # _______________________________
        # check config keywords
        keywords = ['tbeg', 'tend', 'sampling_rate',
                    'station_latitude', 'station_longitude',
                    'baz_step', 'baz_win_sec', 'baz_win_sec_overlap']

        for key in keywords:
            if key not in sixdegrees.attributes(self):
                print(f" -> {key} is missing in config!\n")
                return

        # _______________________________
        # prepare streams
        ACC = sixdegrees.get_translation(self).copy()
        ROT = sixdegrees.get_rotation(self).copy()

        # get amount of samples for data
        n_samples = len(ROT.select(channel="*Z")[0])

        # adjust overlaps
        overlap = self.baz_win_sec_overlap / self.baz_win_sec

        # prepare time windows for loop
        n_windows = n_samples // (int(self.sampling_rate * self.baz_win_sec))

        # prepare backazimuths for loop
        backazimuths = linspace(0, 360 - self.baz_step, int(360 / self.baz_step))

        # prepare data array
        corrbaz = ones([backazimuths.size, n_windows])*nan

        degrees = []
        windows = []

        bazs = ones(n_windows)*nan

        # _______________________________
        # backazimuth estimation with Love or Rayleigh waves
        # loop over backazimuth degrees
        for i_deg in range(0, len(backazimuths)):

            degrees.append(i_deg)

            # loop over time windows
            for i_win in range(0, n_windows):

                if i_deg == 0:
                    windows.append(i_win)

                # infer indices
                idx1 = int(self.sampling_rate * self.baz_win_sec * i_win)
                idx2 = int(self.sampling_rate * self.baz_win_sec * (i_win + 1))

                # add overlap
                if i_win > 0 and i_win < n_windows:
                    idx1 = int(idx1 - overlap * self.baz_win_sec * self.sampling_rate)
                    idx2 = int(idx2 + overlap * self.baz_win_sec * self.sampling_rate)

                # prepare traces according to selected wave type
                if wave_type.lower() == "love":

                    if self.verbose and i_deg == 0 and i_win == 0:
                        print(f"\n -> using {wave_type} waves for BAz estimation ...")

                    # rotate NE to RT
                    R, T = rotate_ne_rt(ACC.select(channel='*N')[0].data,
                                        ACC.select(channel='*E')[0].data,
                                        backazimuths[i_deg]
                                       )

                    # compute correlation for backazimuth
                    ccorr = correlate(ROT.select(channel="*Z")[0][idx1:idx2],
                                      T[idx1:idx2],
                                      0, demean=True, normalize='naive', method='fft'
                                     )

                    # get maximum correlation
                    xshift, cc_max = xcorr_max(ccorr)

                    if xshift != 0:
                        print(f" -> maximal cc not a shift=0: shift={xshift} | cc={cc_max}")

                elif wave_type.lower() == "rayleigh":

                    if self.verbose and i_deg == 0 and i_win == 0:
                        print(f"\n -> using {wave_type} waves for BAz estimation ...")

                    # rotate NE to RT
                    R, T = rotate_ne_rt(ROT.select(channel='*N')[0].data,
                                        ROT.select(channel='*E')[0].data,
                                        backazimuths[i_deg]
                                       )

                    # compute correlation for backazimuth
                    ccorr = correlate(ACC.select(channel="*Z")[0][idx1:idx2],
                                      T[idx1:idx2],
                                      0, demean=True, normalize='naive', method='fft'
                                     )

                    # get maximum correlation
                    xshift, cc_max = xcorr_max(ccorr)

                    if xshift != 0:
                        print(f" -> maximal cc not a shift=0: shift={xshift} | cc={cc_max}")

                elif wave_type.lower() == "tangent":

                    # no grid search, no degrees loop required
                    if i_deg > 0:
                        continue

                    # prepare data
                    dat = (zeros((len(rot_n[idx1:idx2]), 2)))
                    dat[:, 0] = ROT.select(channel='*E')[0].data[idx1:idx2]
                    dat[:, 1] = ROT.select(channel='*N')[0].data[idx1:idx2]

                    # compute covariance
                    covar = cov(dat, rowvar=False)

                    # get dominant eigenvector
                    Cprime, Q = eigh(covar, UPLO='U')

                    # sorting
                    loc = argsort(abs(Cprime))[::-1]

                    # formating
                    Q = Q[:, loc]

                    # get backazimuth using tangent of eigenvectors
                    baz0 = -arctan((Q[1, 0]/Q[0, 0]))*180/pi

                    # if negative due to tangent .. add 180 degrees
                    if baz0 <= 0:
                        baz0 += 180

                    # remove 180° ambiguity
                    R, T = rotate_ne_rt(ROT.select(channel='*N')[0].data,
                                        ROT.select(channel='*E')[0].data,
                                        baz0
                                       )

                    # correlatet with acceleration
                    ccorr = correlate(ACC.select(channel="*Z")[0][idx1:idx2],
                                      T[idx1:idx2],
                                      0, demean=True, normalize='naive', method='fft'
                                     )

                    # get maximum correlation
                    xshift, cc_max = xcorr_max(ccorr)

                    # if correlation positive add 180 degrees
                    if (cc_max > 0):
                        baz0 += 180

                    cc_max = abs(cc_max)

                    # ## add new values to array
                    # if abs(cc_max) > cc_thres:
                    #     baz[j] = baz0
                    #     ccor[j] = abs(corr_baz)

                else:
                    print(f" -> unknown wave type: {wave_type}!")

                corrbaz[i_deg, i_win] = cc_max

                if wave_type.lower() == "tangent":
                    bazs[i_win] = baz0

        # extract maxima
        if wave_type.lower() == "tangent":
            maxbaz = bazs
            maxcorr = corrbaz[0, :]
        else:
            maxbaz = array([backazimuths[corrbaz[:, l1].argmax()] for l1 in range(0, n_windows)])
            maxcorr = array([max(corrbaz[:, l1]) for l1 in range(0, n_windows)])

        # create mesh grid
        t_win = arange(0, self.baz_win_sec*n_windows+self.baz_win_sec, self.baz_win_sec)
        t_win_center = t_win[:-1]+self.baz_win_sec/2
        grid = meshgrid(t_win, backazimuths)

        # add one element for axes
        windows.append(windows[-1]+1)
        degrees.append(degrees[-1]+self.baz_step)

        # add results to object
        if wave_type.lower() == "love":
            self.baz_grid_love = corrbaz
            self.baz_degrees_love = degrees
            self.baz_windows_love = windows
            self.baz_corr_love = maxcorr
            self.baz_max_love = maxbaz
            self.baz_times_love = t_win_center

        elif wave_type.lower() == "rayleigh":
            self.baz_grid_rayleigh = corrbaz
            self.baz_degrees_rayleigh = degrees
            self.baz_windows_ralyeigh = windows
            self.baz_corr_rayleigh = maxcorr
            self.baz_max_rayleigh = maxbaz
            self.baz_times_rayleigh = t_win_center

        elif wave_type.lower() == "tangent":
            self.baz_grid_tangent = corrbaz
            self.baz_degrees_tangent = degrees
            self.baz_windows_tangent = windows
            self.baz_corr_tangent = maxcorr
            self.baz_max_tangent = maxbaz
            self.baz_times_tangent = t_win_center

        if out:
            # _______________________________
            # prepare output
            output = {}

            output['baz_mesh'] = grid
            output['baz_corr'] = corrbaz
            output['acc'] = ACC
            output['rot'] = ROT
            output['cc_max_t'] = t_win_center
            output['cc_max_y'] = maxbaz
            output['cc_max'] = maxcorr

            return output

    @staticmethod
    def get_fft(arr, dt, window=None):

        '''
        Calculating a simple 1D FastFourierSpectrum of a time series.

        >>> frequencies, spectrum, phase = __fft(signal_in, dt ,window=None,normalize=None)
        '''

        from scipy.fft import fft, fftfreq, fftshift
        from scipy import signal
        from numpy import angle, imag

        # determine length of the input time series
        n = int(len(arr))

        # calculate spectrum (with or without window function applied to time series)
        if window is not None:
            win = signal.get_window(window, n);
            spectrum = fft(arr * win)
        else:
            spectrum = fft(arr)

        # calculate frequency array
        frequencies = fftfreq(n, d=dt)

        # calculate amplitude spectrum
        magnitude = abs(spectrum) * 2.0 / n

        # calculate phase spectrum
        phase = angle(spectrum, deg=False)

        return frequencies[0:n//2], magnitude[0:n//2], phase[0:n//2]

    @staticmethod
    def plot_waveform_cc(rot0, acc0, baz, fmin, fmax, distance=None, twin_sec=5, twin_overlap=0.5):

        from obspy.signal.cross_correlation import correlate
        from obspy.signal.rotate import rotate_ne_rt
        from numpy import linspace, ones
        import matplotlib.pyplot as plt

        def __cross_correlation_windows(arr1, arr2, dt, Twin, overlap=0, lag=0, demean=True, plot=False):

            from obspy.signal.cross_correlation import correlate, xcorr_max
            from numpy import arange, array, roll

            N = len(arr1)
            n_interval = int(Twin/dt)
            n_overlap = int(overlap*Twin/dt)

            # time = arange(0, N*dt, dt)

            times, samples = [], []
            n1, n2 = 0, n_interval
            while n2 <= N:
                samples.append((n1, n2))
                times.append(int(n1+(n2-n1)/2)*dt)
                n1 = n1 + n_interval - n_overlap
                n2 = n2 + n_interval - n_overlap

            cc = []
            for _n, (n1, n2) in enumerate(samples):

                _arr1 = roll(arr1[n1:n2], lag)
                _arr2 = arr2[n1:n2]
                ccf = correlate(_arr1, _arr2, 0, demean=demean, normalize='naive', method='fft')
                shift, val = xcorr_max(ccf, abs_max=False)
                cc.append(val)

            return array(times), array(cc)


        rot = rot0.copy()
        acc = acc0.copy()

        Nrow, Ncol = 3, 1

        fig, ax = plt.subplots(Nrow, Ncol, figsize=(15, 6), sharex=True)

        acc_scaling, acc_unit = 1e3, f"mm/s$^2$"
        rot_scaling, rot_unit = 1e6, f"$\mu$rad/s"

        lw = 1

        font = 12

        acc_z = acc.select(channel="*Z")[0].data
        rot_z = rot.select(channel="*Z")[0].data

        acc_r, acc_t = rotate_ne_rt(acc.select(channel="*N")[0].data, acc.select(channel="*E")[0].data, baz)
        rot_r, rot_t = rotate_ne_rt(rot.select(channel="*N")[0].data, rot.select(channel="*E")[0].data, baz)

        rot_z*=rot_scaling
        rot_r*=rot_scaling
        rot_t*=rot_scaling

        acc_z*=acc_scaling
        acc_r*=acc_scaling
        acc_t*=acc_scaling

        acc_z_max = max([abs(min(acc_z)), abs(max(acc_z))])
        acc_r_max = max([abs(min(acc_r)), abs(max(acc_r))])
        acc_t_max = max([abs(min(acc_t)), abs(max(acc_t))])

        rot_z_max = max([abs(min(rot_z)), abs(max(rot_z))])
        rot_r_max = max([abs(min(rot_r)), abs(max(rot_r))])
        rot_t_max = max([abs(min(rot_t)), abs(max(rot_t))])

        dt = rot[0].stats.delta

        rot0 ,acc0, rot0_lbl, acc0_lbl = rot_z, acc_t, "ROT-Z", "ACC-T"
        rot1 ,acc1, rot1_lbl, acc1_lbl = rot_t, acc_z, "ROT_T", "ACC-Z"
        rot2 ,acc2, rot2_lbl, acc2_lbl = rot_z, -acc_r, "ROT-Z", "-1xACC-R"

        tt0, cc0 = __cross_correlation_windows(rot0 ,acc0, dt, twin_sec, overlap=twin_overlap, lag=0, demean=True)
        tt1, cc1 = __cross_correlation_windows(rot1, acc1, dt, twin_sec, overlap=twin_overlap, lag=0, demean=True)
        tt2, cc2 = __cross_correlation_windows(rot2, acc2, dt, twin_sec, overlap=twin_overlap, lag=0, demean=True)

        cmap = plt.get_cmap("coolwarm", 12)


        ax[0].plot(rot.select(channel="*Z")[0].times(), rot0, label=rot0_lbl, color="tab:red", lw=lw, zorder=3)
        ax00 = ax[0].twinx()
        ax00.plot(acc.select(channel="*Z")[0].times(), acc0, label=acc0_lbl, color="black", lw=lw)
        ax01 = ax[0].twinx()
        cm = ax01.scatter(tt0, ones(len(tt0))*-0.9, c=cc0, alpha=abs(cc0), cmap=cmap, label="")

        ax[0].set_ylim(-rot_z_max, rot_z_max)
        ax00.set_ylim(-acc_t_max, acc_t_max)
        ax01.set_ylim(-1, 1)
        ax01.yaxis.set_visible(False)

        ax[1].plot(rot.select(channel="*N")[0].times(), rot1, label=rot1_lbl, color="tab:red", lw=lw, zorder=3)
        ax11 = ax[1].twinx()
        ax11.plot(acc.select(channel="*Z")[0].times(), acc1, label=acc1_lbl, color="black", lw=lw)
        ax12 = ax[1].twinx()
        ax12.scatter(tt1, ones(len(tt1))*-0.9, c=cc1, alpha=abs(cc1), cmap=cmap, label="")

        ax[1].set_ylim(-rot_t_max, rot_t_max)
        ax11.set_ylim(-acc_z_max, acc_z_max)
        ax12.set_ylim(-1, 1)
        ax12.yaxis.set_visible(False)

        ax[2].plot(rot.select(channel="*N")[0].times(), rot2, label=rot2_lbl, color="tab:red", lw=lw, zorder=3)
        ax22 = ax[2].twinx()
        ax22.plot(acc.select(channel="*Z")[0].times(), acc2, label=acc2_lbl, color="black", lw=lw)
        ax23 = ax[2].twinx()
        ax23.scatter(tt2, ones(len(tt2))*-0.9, c=cc2, alpha=abs(cc2), cmap=cmap, label="")

        ax[2].set_ylim(-rot_z_max, rot_z_max)
        ax22.set_ylim(-acc_r_max, acc_r_max)
        ax23.set_ylim(-1, 1)
        ax23.yaxis.set_visible(False)

        cc0 = round(correlate(rot0, acc0, 0, demean=True, normalize='naive', method='auto')[0], 2)
        cc1 = round(correlate(rot1, acc1, 0, demean=True, normalize='naive', method='auto')[0], 2)
        cc2 = round(correlate(rot2, acc2, 0, demean=True, normalize='naive', method='auto')[0], 2)
        cc = [cc0, cc1, cc2]

        ## sync twinx
        ax[0].set_yticks(linspace(ax[0].get_yticks()[0], ax[0].get_yticks()[-1], len(ax[0].get_yticks())))
        ax00.set_yticks(linspace(ax00.get_yticks()[0], ax00.get_yticks()[-1], len(ax[0].get_yticks())))

        ax[1].set_yticks(linspace(ax[1].get_yticks()[0], ax[1].get_yticks()[-1], len(ax[1].get_yticks())))
        ax11.set_yticks(linspace(ax11.get_yticks()[0], ax11.get_yticks()[-1], len(ax[1].get_yticks())))

        ax[2].set_yticks(linspace(ax[2].get_yticks()[0], ax[2].get_yticks()[-1], len(ax[2].get_yticks())))
        ax22.set_yticks(linspace(ax22.get_yticks()[0], ax22.get_yticks()[-1], len(ax[2].get_yticks())))

        for i in range(3):
            ax[i].legend(loc=1, ncols=4)
            ax[i].grid(which="both", alpha=0.5)
            ax[i].set_ylabel(f"$\Omega$ ({rot_unit})", fontsize=font)
            ax[i].text(0.05, 0.9, f"CC={cc[i]}", ha='left', va='top', transform=ax[i].transAxes, fontsize=font-1)

        for _ax in [ax00, ax11, ax22]:
            _ax.legend(loc=4)
            _ax.set_ylabel(f"$a$ ({acc_unit})", fontsize=font)

        ax[2].set_xlabel("Time (s)", fontsize=font)

        tbeg = acc[0].stats.starttime
        ax[0].set_title(f"{tbeg.date} {str(tbeg.time).split('.')[0]} UTC  |  f = {fmin}-{fmax} Hz  |  BAz = {round(baz, 1)}°  |  ED = {round(distance/1000,1)} km  |  T = {twin_sec}s ({int(100*twin_overlap)}%)")

        cax = ax[Nrow-1].inset_axes([0.8, -0.35, 0.2, 0.1], transform=ax[Nrow-1].transAxes)
        cb = plt.colorbar(cm, cax=cax, shrink=0.4, location='bottom', orientation='horizontal')
        cm.set_clim(-1, 1)
        cb.set_label("Cross-Correlation", fontsize=font, loc="left", labelpad=-43, color="black", backgroundcolor="w")

        plt.show();
        return fig


In [7]:
from obspy import UTCDateTime

config = {}

config['project'] = "rolode/"

config['verbose'] = True

config['tbeg'] = UTCDateTime("2024-09-09T00:00:00")
config['tend'] = UTCDateTime("2024-09-09T01:00:00")

config['tra_seed'] = 'GR.FUR..BH*' # seed of translational data
config['rot_seed'] = 'BW.ROMY.30.BJ*' # seed of rotational data

config['station_lon'] = 11.275501
config['station_lat'] = 48.162941

# specify response information file
config['path_to_inv_tra'] = data_path+'stationxml_ringlaser/station_GR_FUR.xml' #seismometer
config['path_to_inv_rot'] = data_path+'stationxml_ringlaser/station_BW_ROMY.xml' #blueSeis

config['path_to_sds_rot'] = archive_path+"temp_archive/"
config['path_to_sds_tra'] = bay_path+"mseed_online/archive/"

test = sixdegrees(config)

In [ ]:
test.load_data(t1=config['tbeg'], t2=config['tend'])

/home/andbro/ontap-ffb-bay200/mseed_online/archive/ GR.FUR..BH*


In [ ]:
test.trim_stream()

In [ ]:
test.st

In [ ]:
test.fmin = 0.1
test.fmax = 0.5

In [ ]:
test.plot_waveform_cc(test.get_rotation(),
                      test.get_translation(),
                      300,
                      test.fmin, test.fmax,
                      distance=0,
                      twin_sec=30,
                      twin_overlap=0.5
                     );


In [ ]:
keywords = ['tbeg', 'tend', 'sampling_rate',
            'station_latitude', 'station_longitude',
            'baz_step', 'baz_win_sec', 'baz_win_sec_overlap']


In [ ]:
test.sampling_rate = 20
test.baz_step = 1
test.baz_win_sec = 30
test.baz_win_sec_overlap = 10


In [ ]:
test.compute_backazimuth(wave_type="rayleigh")

In [ ]:
test.compute_backazimuth(wave_type="love")

In [ ]:
test.compute_backazimuth(wave_type="tangent")

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(test.baz_times_love, test.baz_max_love)
plt.scatter(test.baz_times_rayleigh, test.baz_max_rayleigh)
plt.scatter(test.baz_times_tangent, test.baz_max_tangent)

In [ ]:
import numpy as np

plt.pcolormesh(test.baz_windows_love, test.baz_degrees_love, test.baz_grid_love)
